In [114]:
# ideas
# two different models for player 0 moving and player 1 moving

from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [2]:
pd.options.display.max_columns = None

In [3]:
game_states_data_test = pd.read_csv('game_states_data_test.csv')

In [4]:
game_states_data_training = pd.read_csv('game_states_data_training.csv')
game_states_data_training.head()

,instance_id,frag,map_id,game_mode,moving_player,player0_colour,player1_colour,player0_AI,player1_AI,hp_player0_unit1,hp_player0_unit2,hp_player0_unit3,hp_player0_unit4,hp_player1_unit1,hp_player1_unit2,hp_player1_unit3,hp_player1_unit4,position_x_player0_unit1,position_x_player0_unit2,position_x_player0_unit3,position_x_player0_unit4,position_x_player1_unit1,position_x_player1_unit2,position_x_player1_unit3,position_x_player1_unit4,position_y_player0_unit1,position_y_player0_unit2,position_y_player0_unit3,position_y_player0_unit4,position_y_player1_unit1,position_y_player1_unit2,position_y_player1_unit3,position_y_player1_unit4,player0_unit1_type,player0_unit2_type,player0_unit3_type,player0_unit4_type,player1_unit1_type,player1_unit2_type,player1_unit3_type,player1_unit4_type
0,1,True,Map03,Domination,1,#0000FF,#FF0000,False,False,100,36,140,0,20,0,18,0,13.128258,-5.072649,-9.833621,-6.183261,-6.545395,-2.876717,7.301014,2.736246,4.488010,1.017117,3.776405,2.023968,-0.979725,-5.986560,-5.756070,-5.031605,Assault,Assault,Heavy,Scout,Assault,Assault,Heavy,Scout
1,2,False,Map05,Domination,0,#FF0000,#00FF00,True,True,60,0,60,60,140,60,100,100,-5.567025,-0.101519,-0.355085,-7.342392,-3.011566,0.380839,3.170114,6.472121,1.275000,11.982550,-10.915156,4.236153,1.018507,6.970410,-3.966517,-7.417881,Scout,Scout,Scout,Scout,Heavy,Scout,Assault,Assault
2,3,False,Map08,Devastator,0,#FF0000,#00FF00,True,True,70,140,140,0,0,0,100,46,-7.465430,-4.036615,10.611968,0.870345,-5.867641,-5.825762,-1.137856,12.664976,-6.357566,3.520604,-13.028067,-12.942177,-10.912375,4.003931,-11.172128,-7.272110,Support,Heavy,Heavy,Heavy,Assault,Assault,Assault,Assault
3,4,False,Map17,Domination,1,#FF0000,#00FF00,True,True,0,0,0,100,0,0,100,140,-7.163575,12.378940,0.610483,9.084709,0.112198,-10.120893,-7.943638,-14.639385,5.413563,6.929533,-0.110481,5.084709,0.089166,-2.091748,-5.143735,-0.577571,Assault,Assault,Assault,Assault,Scout,Support,Assault,Heavy
4,5,True,Map22,Domination,0,#FF0000,#00FF00,True,True,60,140,60,140,20,100,100,100,-5.403627,-8.297834,9.850985,-10.421861,-10.393658,-0.311284,8.106512,5.091607,-6.588476,-3.373036,-0.156573,3.173592,-3.083063,6.125000,2.914917,2.118116,Scout,Heavy,Scout,Heavy,Heavy,Assault,Assault,Assault


In [131]:
print(game_states_data_training['frag'].value_counts())
print(game_states_data_training['game_mode'].value_counts())
print(game_states_data_training['moving_player'].value_counts())

print(game_states_data_test['frag'].value_counts())
print(game_states_data_test['game_mode'].value_counts())
print(game_states_data_test['moving_player'].value_counts())

False    40410
True      9590
Name: frag, dtype: int64
Domination    29660
Devastator    20340
Name: game_mode, dtype: int64
0    25696
1    24304
Name: moving_player, dtype: int64
Series([], Name: frag, dtype: int64)
Domination    5861
Devastator    4139
Name: game_mode, dtype: int64
0    5069
1    4931
Name: moving_player, dtype: int64


In [5]:
game_states_data_training.describe()

,instance_id,moving_player,hp_player0_unit1,hp_player0_unit2,hp_player0_unit3,hp_player0_unit4,hp_player1_unit1,hp_player1_unit2,hp_player1_unit3,hp_player1_unit4,position_x_player0_unit1,position_x_player0_unit2,position_x_player0_unit3,position_x_player0_unit4,position_x_player1_unit1,position_x_player1_unit2,position_x_player1_unit3,position_x_player1_unit4,position_y_player0_unit1,position_y_player0_unit2,position_y_player0_unit3,position_y_player0_unit4,position_y_player1_unit1,position_y_player1_unit2,position_y_player1_unit3,position_y_player1_unit4
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,25000.500000,0.486080,64.395060,64.610480,85.960020,55.681700,65.918760,67.038240,86.848440,58.812360,-1.913848,0.779050,-0.777698,1.020953,1.032940,3.615924,2.874194,3.718101,-0.264370,0.855062,-0.221130,-1.077481,0.691452,-0.155636,-1.428473,-1.188604
std,14433.901067,0.499811,46.840244,47.237522,52.344052,43.683389,45.830947,46.216319,51.613061,43.062093,6.908333,7.594172,7.591717,7.815768,6.560436,7.102644,6.623827,7.366739,6.525855,6.398245,6.437146,6.527839,7.008903,7.019888,7.199114,7.004185
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-18.189234,-18.126446,-18.604462,-20.205133,-18.590970,-19.396637,-19.602653,-20.352852,-16.725710,-16.671610,-16.456327,-16.711794,-16.697412,-17.287556,-17.690884,-17.714697
25%,12500.750000,0.000000,0.000000,0.000000,60.000000,0.000000,6.000000,4.000000,60.000000,5.000000,-7.122645,-4.706121,-7.029804,-5.361141,-3.764173,-0.545678,-1.203205,-0.790837,-4.443597,-3.738889,-4.909196,-6.711933,-4.675685,-5.358068,-7.297277,-6.682206
50%,25000.500000,0.000000,70.000000,70.000000,100.000000,60.000000,70.000000,70.000000,100.000000,60.000000,-3.037777,-0.071294,-1.344546,0.716799,1.652753,4.840132,3.851756,4.902444,0.184329,0.955398,0.012804,-0.586075,1.083495,0.404670,-1.235838,-1.358622
75%,37500.250000,1.000000,100.000000,100.000000,140.000000,70.000000,100.000000,100.000000,140.000000,94.000000,2.821763,5.435797,5.390935,7.166426,6.187472,8.419000,7.826073,8.891031,4.260918,4.977633,4.284558,3.731799,6.500000,5.319068,4.439662,4.259862
max,50000.000000,1.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,18.969131,20.744686,19.608790,21.548782,18.976147,18.970566,18.935515,22.510527,18.482018,18.793589,18.052328,16.994791,18.044231,18.742983,17.296816,18.541710


In [6]:
game_states_data_test.describe()

,instance_id,frag,moving_player,hp_player0_unit1,hp_player0_unit2,hp_player0_unit3,hp_player0_unit4,hp_player1_unit1,hp_player1_unit2,hp_player1_unit3,hp_player1_unit4,position_x_player0_unit1,position_x_player0_unit2,position_x_player0_unit3,position_x_player0_unit4,position_x_player1_unit1,position_x_player1_unit2,position_x_player1_unit3,position_x_player1_unit4,position_y_player0_unit1,position_y_player0_unit2,position_y_player0_unit3,position_y_player0_unit4,position_y_player1_unit1,position_y_player1_unit2,position_y_player1_unit3,position_y_player1_unit4
count,10000.00000,0.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,55000.50000,NaN,0.493100,63.652800,64.587000,86.464100,55.402800,64.200900,67.286900,86.235000,58.903700,-1.877465,0.773935,-0.561378,1.334828,1.102203,3.749876,2.913107,3.771153,-0.115404,0.857997,-0.309735,-0.967715,0.761339,0.086915,-1.396968,-0.772683
std,2886.89568,NaN,0.499977,47.773942,48.209912,52.237529,44.613296,45.630746,46.997194,52.619565,43.477388,6.930699,7.625565,7.628924,7.728243,6.696973,6.959662,6.488164,7.365589,6.307688,6.366522,6.487798,6.557180,6.986840,6.903939,7.298934,7.070713
min,50001.00000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.544956,-18.122234,-18.124960,-18.044678,-18.341543,-17.875000,-18.883690,-20.090740,-16.487595,-15.858482,-16.722094,-16.724260,-16.186256,-16.220690,-17.206245,-17.208950
25%,52500.75000,NaN,0.000000,0.000000,0.000000,60.000000,0.000000,0.000000,8.000000,60.000000,0.000000,-7.086421,-4.661408,-6.894203,-5.026492,-3.784969,-0.387788,-1.085007,-0.829198,-4.177029,-3.818068,-5.230386,-6.479919,-4.687416,-5.215528,-7.475000,-6.236832
50%,55000.50000,NaN,0.000000,70.000000,70.000000,100.000000,60.000000,70.000000,70.000000,100.000000,60.000000,-3.011155,-0.027676,-1.099039,1.640239,1.930721,4.897627,3.874751,4.994250,0.232755,0.866185,-0.012990,-0.643422,1.191882,0.584275,-1.207970,-0.671052
75%,57500.25000,NaN,1.000000,100.000000,100.000000,140.000000,70.000000,100.000000,100.000000,140.000000,100.000000,2.692228,5.390310,5.569179,7.362357,6.247814,8.421898,7.658553,8.957903,4.046183,4.876832,4.247876,3.796300,6.500000,5.575662,4.771732,4.656846
max,60000.00000,NaN,1.000000,1000.000000,1000.000000,1000.000000,1000.000000,140.000000,140.000000,140.000000,140.000000,18.880632,18.970251,18.914967,20.110323,18.954590,18.967615,18.929384,19.208889,17.148480,18.524977,16.543468,16.497812,16.025000,17.288607,17.753927,16.624983


In [104]:
# dropping features without much added value
# engineering new features

X = game_states_data_training

# basic columns to drop
X = X.drop('frag', axis=1)
X = X.drop('instance_id', axis=1)

X_player0 = X[[c for c in X.columns if 'player0' in c]]
X_player1 = X[[c for c in X.columns if 'player1' in c]]

# engineering new features
X['player0_no_assault'] = X_player0.apply(lambda x: x.str.contains('Assault', regex=False).sum(), axis=1)
X['player0_no_heavy'] = X_player0.apply(lambda x: x.str.contains('Heavy', regex=False).sum(), axis=1)
X['player0_no_scout'] = X_player0.apply(lambda x: x.str.contains('Scout', regex=False).sum(), axis=1)
X['player0_no_support'] = X_player0.apply(lambda x: x.str.contains('Support', regex=False).sum(), axis=1)
X['player1_no_assault'] = X_player1.apply(lambda x: x.str.contains('Assault', regex=False).sum(), axis=1)
X['player1_no_heavy'] = X_player1.apply(lambda x: x.str.contains('Heavy', regex=False).sum(), axis=1)
X['player1_no_scout'] = X_player1.apply(lambda x: x.str.contains('Scout', regex=False).sum(), axis=1)
X['player1_no_support'] = X_player1.apply(lambda x: x.str.contains('Support', regex=False).sum(), axis=1)

# further columns to drop
X = X.drop(['map_id', 
            'player0_colour', 'player1_colour',
            'player0_unit1_type', 'player0_unit2_type', 'player0_unit3_type', 'player0_unit4_type',
            'player1_unit1_type', 'player1_unit2_type', 'player1_unit3_type', 'player1_unit4_type'], axis=1)

final_X = X.copy()

In [138]:
# model training

X = final_X

numerical_columns = [c for c in X.columns if (X[c].dtype == 'float64' or X[c].dtype == 'int64')]
categorical_columns = [c for c in X.columns if X[c].dtype == 'object']

y = game_states_data_training['frag'].astype(int)

ct = make_column_transformer(
    (StandardScaler(), numerical_columns),
    (OneHotEncoder(), categorical_columns),
    remainder='drop')

X_t = ct.fit_transform(X)

X = pd.DataFrame(
    X_t,
    columns=ct.get_feature_names_out(X.columns)
)

print(X.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

gbm_param_grid = {
    'n_estimators': [10, 20, 50, 100, 200, 500],
    'max_depth': [2, 3, 4, 5, 6, 7, 8],
    'colsample_bytree': [0.5, 0.66, 0.83, 1],
    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
    'subsample': [0.5, 0.66, 0.83, 1],
    'min_child_weight': [0, 1, 2, 3, 4, 5],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

gbm = xgb.XGBClassifier()

# set up RandomizedSearchCV()
randomized_mse = RandomizedSearchCV(estimator=gbm, 
                        param_distributions=gbm_param_grid,
                        scoring='roc_auc',
                        cv=3,
                        n_iter=3,
                        verbose=4,
                        return_train_score=True)      

# fit the model
randomized_mse.fit(x_train, y_train)               

print("Best parameters found: ", randomized_mse.best_params_)

for el in zip(X.columns, randomized_mse.best_estimator_.feature_importances_):
    print(el)

y_pred = randomized_mse.predict_proba(x_test)[:, 1]

print(np.mean(y_pred))
print(np.median(y_pred))
print(np.std(y_pred))

print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

(50000, 35)
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END colsample_bytree=0.83, gamma=0, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=20, subsample=0.66;, score=(train=0.861, test=0.707) total time=   0.5s
[CV 2/3] END colsample_bytree=0.83, gamma=0, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=20, subsample=0.66;, score=(train=0.868, test=0.695) total time=   0.5s
[CV 3/3] END colsample_bytree=0.83, gamma=0, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=20, subsample=0.66;, score=(train=0.868, test=0.708) total time=   0.5s
[CV 1/3] END colsample_bytree=1, gamma=0.1, learning_rate=0.5, max_depth=6, min_child_weight=5, n_estimators=100, subsample=0.83;, score=(train=0.977, test=0.702) total time=   2.6s
[CV 2/3] END colsample_bytree=1, gamma=0.1, learning_rate=0.5, max_depth=6, min_child_weight=5, n_estimators=100, subsample=0.83;, score=(train=0.974, test=0.699) total time=   2.5s
[CV 3/3] END colsam

In [139]:
# modify the test data the same way the training data was modified and predict results

X = game_states_data_test

# basic columns to drop
X = X.drop('frag', axis=1)
X = X.drop('instance_id', axis=1)

X_player0 = X[[c for c in X.columns if 'player0' in c]]
X_player1 = X[[c for c in X.columns if 'player1' in c]]

# engineering new features
X['player0_no_assault'] = X_player0.apply(lambda x: x.str.contains('Assault', regex=False).sum(), axis=1)
X['player0_no_heavy'] = X_player0.apply(lambda x: x.str.contains('Heavy', regex=False).sum(), axis=1)
X['player0_no_scout'] = X_player0.apply(lambda x: x.str.contains('Scout', regex=False).sum(), axis=1)
X['player0_no_support'] = X_player0.apply(lambda x: x.str.contains('Support', regex=False).sum(), axis=1)
X['player1_no_assault'] = X_player1.apply(lambda x: x.str.contains('Assault', regex=False).sum(), axis=1)
X['player1_no_heavy'] = X_player1.apply(lambda x: x.str.contains('Heavy', regex=False).sum(), axis=1)
X['player1_no_scout'] = X_player1.apply(lambda x: x.str.contains('Scout', regex=False).sum(), axis=1)
X['player1_no_support'] = X_player1.apply(lambda x: x.str.contains('Support', regex=False).sum(), axis=1)

# further columns to drop
X = X.drop(['map_id', 
            'player0_colour', 'player1_colour',
            'player0_unit1_type', 'player0_unit2_type', 'player0_unit3_type', 'player0_unit4_type',
            'player1_unit1_type', 'player1_unit2_type', 'player1_unit3_type', 'player1_unit4_type'], axis=1)

numerical_columns = [c for c in X.columns if (X[c].dtype == 'float64' or X[c].dtype == 'int64')]
categorical_columns = [c for c in X.columns if X[c].dtype == 'object']

X_t = ct.transform(X)

X = pd.DataFrame(
    X_t,
    columns=ct.get_feature_names_out(X.columns)
)

y_pred = randomized_mse.predict_proba(X)[:, 1]

with open('res.txt', 'w') as f:
    for y in y_pred:
        f.write(f'{y}\n')
        
print(np.mean(y_pred))
print(np.median(y_pred))
print(np.std(y_pred))

0.19152446
0.14959796
0.16494894
